In [1]:
import pandas as pd
import re

In [2]:
class binding_protein_filter:
    
    def __init__(self, data_path, binding_protein_path, predict_result_path):
        self.binding_protein_path = binding_protein_path
        self.predict_result_path = predict_result_path
        self.input_data_csv = pd.read_csv(data_path)

    def binding_protein_filter(self):
        input_data = self.input_data_csv
        BP = pd.read_csv(self.binding_protein_path,header=None)
        BP = BP.iloc[::2,0,]
        BP = BP.reset_index(drop=True)
        
        # 위에서 불러온 파일 중 ENSP 부분 필터링 후 저장
        column_list = list(input_data.columns)
        extracted_ids = [re.search(r'(>ENSP\d+)', item).group(1) for item in column_list]
        
        BP_binding = []
        
        for i in BP:
            for j in extracted_ids:
                if j in i:
                    BP_binding.append(i)

        # 불러온 파일 중 binding_protein만 남김
        ok_list = []
        for l in range(len(input_data.columns)):
            for k in BP_binding:
                if k in input_data.columns[l]:
                    ok_list.append(input_data.columns[l])

        binding_protein = input_data[ok_list]
        
        return binding_protein
    
    def predict_right_filter(self,filtered_binding_protein):
        # 실제 값이 1이고 예측 결과가 정확한 경우만 필터
        # CSV 파일에서 데이터프레임 읽기
        predict_df = pd.read_csv(predict_result_path)

        # 필터링된 결과를 저장할 리스트 초기화
        keep_list = []

        # 데이터프레임을 순회하면서 조건에 맞는 데이터 추출
        for i in range(len(predict_df)):
            if predict_df.loc[i, 'class'] == 1 and predict_df.loc[i, 'class'] == predict_df.loc[i, 'prediction']:
                # 리스트 대신 값 자체를 keep_list에 추가
                keep_list.append(predict_df.loc[i, 'ID'])
                
        binding_correct_protein = pd.DataFrame()

        for j in range(len(filtered_binding_protein.columns)):
            for k in keep_list:
                if k in filtered_binding_protein.columns[j]:
                    binding_correct_protein = pd.concat([binding_correct_protein, filtered_binding_protein.iloc[:,j]],axis=1)
        
        return binding_correct_protein
    
    def column_name_modification(self, binding_correct_protein):
        # 데이터 프레임을 보기 좋은 형태로 변경
        new_data = [] 

        # 리스트의 각 요소를 순회하면서 정규 표현식을 사용하여 변환
        for item in binding_correct_protein:
            # 'tensor' 표현을 제거하고 Delta 값을 추출
            new_item = re.sub(r'tensor\(\[([-\d.]+)\], dtype=torch.float64\)', r'\1', item)
            # 중복된 'Delta:' 표현을 하나로 줄임
            new_item = re.sub(r'\(Delta: \(Delta: ([-\d.]+)\)\)', r'(Delta: \1)', new_item)
            new_data.append(new_item)

        binding_correct_protein.columns = new_data
        
        return binding_correct_protein
    

In [38]:
kind_of_work = 'RBP'
iteration = 2
Data_path = f"/Users/csb/Desktop/{kind_of_work}_captum/{kind_of_work}{iteration}_attribution_score.csv"
Binding_protein_path = "/Users/csb/Desktop/Project/DNA-RNA-binding/RNA-DNA machine learning/DNA-RNA-BP-ProteinFASTA/351_RNAbinding_Protein.txt"
predict_result_path = f'/Users/csb/Desktop/{kind_of_work}_captum/result_of_{kind_of_work}{iteration}.csv'

In [39]:
test = pd.read_csv(Data_path)
test

,">ENSP00000436580 (Delta: tensor([0.2864], dtype=torch.float64))",">ENSP00000357869 (Delta: tensor([0.0628], dtype=torch.float64))",">ENSP00000400646 (Delta: tensor([-0.3486], dtype=torch.float64))",">ENSP00000396445 (Delta: tensor([0.1646], dtype=torch.float64))",">ENSP00000286692 (Delta: tensor([-0.0050], dtype=torch.float64))",">ENSP00000410242 (Delta: tensor([-0.2813], dtype=torch.float64))",">ENSP00000477443 (Delta: tensor([-0.0776], dtype=torch.float64))",">ENSP00000309270 (Delta: tensor([-0.1064], dtype=torch.float64))",">ENSP00000379364 (Delta: tensor([0.0280], dtype=torch.float64))",">ENSP00000239165 (Delta: tensor([-0.0265], dtype=torch.float64))",...,">ENSP00000417659 (Delta: tensor([-0.1440], dtype=torch.float64))",">ENSP00000343023 (Delta: tensor([-0.4162], dtype=torch.float64))",">ENSP00000308354 (Delta: tensor([0.1958], dtype=torch.float64))",">ENSP00000442304 (Delta: tensor([0.1130], dtype=torch.float64))",">ENSP00000358140 (Delta: tensor([0.0479], dtype=torch.float64))",">ENSP00000297440 (Delta: tensor([-0.0476], dtype=torch.float64))",">ENSP00000239882 (Delta: tensor([-0.2555], dtype=torch.float64))",">ENSP00000460157 (Delta: tensor([0.0969], dtype=torch.float64))",">ENSP00000480140 (Delta: tensor([0.0065], dtype=torch.float64))",">ENSP00000257552 (Delta: tensor([0.4922], dtype=torch.float64))"
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,-0.0022,0.0023,-0.0315,-0.0015,-0.0043,-0.1454,-0.0027,-0.0049,-0.0012,-0.0004,...,0.0173,-0.0379,0.0013,0.0010,0.0075,0.0103,-0.0143,-0.0048,0.0223,-0.0006
2,-0.0088,-0.0003,0.0045,0.0004,-0.0042,-0.0250,0.0028,-0.0005,0.0062,-0.0001,...,0.0007,0.0008,0.0029,0.0000,0.0027,0.0010,0.0017,-0.0006,0.0094,0.0051
3,0.0157,-0.0020,0.0053,0.0008,-0.0068,-0.0079,-0.0001,0.0001,-0.0051,0.0015,...,0.0007,-0.0169,0.0014,0.0015,0.0067,-0.0001,0.0010,-0.0007,-0.0009,0.0060
4,-0.0039,-0.0013,0.0144,-0.0014,-0.0046,-0.0262,0.0103,-0.0011,0.0035,0.0023,...,0.0006,-0.0128,0.0013,0.0014,0.0017,0.0020,-0.0003,-0.0011,-0.0024,0.0063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,...,100.0000,100.0000,100.0000,100.0000,-0.0023,100.0000,100.0000,100.0000,100.0000,100.0000
1022,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,...,100.0000,100.0000,100.0000,100.0000,0.0024,100.0000,100.0000,100.0000,100.0000,100.0000
1023,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,...,100.0000,100.0000,100.0000,100.0000,0.0000,100.0000,100.0000,100.0000,100.0000,100.0000
1024,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,...,100.0000,100.0000,100.0000,100.0000,0.0006,100.0000,100.0000,100.0000,100.0000,100.0000


In [40]:
bp_filter = binding_protein_filter(Data_path, Binding_protein_path,predict_result_path)
b_p = bp_filter.binding_protein_filter()
b_p

,">ENSP00000400646 (Delta: tensor([-0.3486], dtype=torch.float64))",">ENSP00000410242 (Delta: tensor([-0.2813], dtype=torch.float64))",">ENSP00000477443 (Delta: tensor([-0.0776], dtype=torch.float64))",">ENSP00000357459 (Delta: tensor([0.0759], dtype=torch.float64))",">ENSP00000340796 (Delta: tensor([0.0604], dtype=torch.float64))",">ENSP00000355340 (Delta: tensor([0.1131], dtype=torch.float64))",">ENSP00000225388 (Delta: tensor([-0.2702], dtype=torch.float64))",">ENSP00000337736 (Delta: tensor([0.1470], dtype=torch.float64))",">ENSP00000356669 (Delta: tensor([0.0842], dtype=torch.float64))",">ENSP00000265085 (Delta: tensor([-0.0539], dtype=torch.float64))",">ENSP00000445508 (Delta: tensor([1.0364], dtype=torch.float64))",">ENSP00000358140 (Delta: tensor([0.0479], dtype=torch.float64))",">ENSP00000480140 (Delta: tensor([0.0065], dtype=torch.float64))",">ENSP00000257552 (Delta: tensor([0.4922], dtype=torch.float64))"
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,-0.0315,-0.1454,-0.0027,-0.0068,-0.0143,0.0058,0.0541,0.0024,0.0015,0.0343,0.0805,0.0075,0.0223,-0.0006
2,0.0045,-0.0250,0.0028,0.0006,-0.0089,0.0025,0.0119,-0.0055,0.0006,-0.0057,0.0112,0.0027,0.0094,0.0051
3,0.0053,-0.0079,-0.0001,0.0011,0.0011,0.0047,0.0270,-0.0065,0.0009,-0.0015,0.0041,0.0067,-0.0009,0.0060
4,0.0144,-0.0262,0.0103,0.0006,-0.0104,0.0032,0.0086,-0.0030,0.0016,-0.0008,0.0133,0.0017,-0.0024,0.0063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,100.0000,100.0000,100.0000,0.0002,100.0000,100.0000,100.0000,100.0000,0.0006,100.0000,100.0000,-0.0023,100.0000,100.0000
1022,100.0000,100.0000,100.0000,0.0027,100.0000,100.0000,100.0000,100.0000,0.0006,100.0000,100.0000,0.0024,100.0000,100.0000
1023,100.0000,100.0000,100.0000,0.0022,100.0000,100.0000,100.0000,100.0000,0.0005,100.0000,100.0000,0.0000,100.0000,100.0000
1024,100.0000,100.0000,100.0000,-0.0002,100.0000,100.0000,100.0000,100.0000,0.0006,100.0000,100.0000,0.0006,100.0000,100.0000


In [41]:
bc_p = bp_filter.predict_right_filter(b_p)
bc_p

,">ENSP00000400646 (Delta: tensor([-0.3486], dtype=torch.float64))",">ENSP00000410242 (Delta: tensor([-0.2813], dtype=torch.float64))",">ENSP00000477443 (Delta: tensor([-0.0776], dtype=torch.float64))",">ENSP00000357459 (Delta: tensor([0.0759], dtype=torch.float64))",">ENSP00000340796 (Delta: tensor([0.0604], dtype=torch.float64))",">ENSP00000337736 (Delta: tensor([0.1470], dtype=torch.float64))",">ENSP00000265085 (Delta: tensor([-0.0539], dtype=torch.float64))",">ENSP00000445508 (Delta: tensor([1.0364], dtype=torch.float64))",">ENSP00000358140 (Delta: tensor([0.0479], dtype=torch.float64))",">ENSP00000480140 (Delta: tensor([0.0065], dtype=torch.float64))",">ENSP00000257552 (Delta: tensor([0.4922], dtype=torch.float64))"
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,-0.0315,-0.1454,-0.0027,-0.0068,-0.0143,0.0024,0.0343,0.0805,0.0075,0.0223,-0.0006
2,0.0045,-0.0250,0.0028,0.0006,-0.0089,-0.0055,-0.0057,0.0112,0.0027,0.0094,0.0051
3,0.0053,-0.0079,-0.0001,0.0011,0.0011,-0.0065,-0.0015,0.0041,0.0067,-0.0009,0.0060
4,0.0144,-0.0262,0.0103,0.0006,-0.0104,-0.0030,-0.0008,0.0133,0.0017,-0.0024,0.0063
...,...,...,...,...,...,...,...,...,...,...,...
1021,100.0000,100.0000,100.0000,0.0002,100.0000,100.0000,100.0000,100.0000,-0.0023,100.0000,100.0000
1022,100.0000,100.0000,100.0000,0.0027,100.0000,100.0000,100.0000,100.0000,0.0024,100.0000,100.0000
1023,100.0000,100.0000,100.0000,0.0022,100.0000,100.0000,100.0000,100.0000,0.0000,100.0000,100.0000
1024,100.0000,100.0000,100.0000,-0.0002,100.0000,100.0000,100.0000,100.0000,0.0006,100.0000,100.0000


In [42]:
clmn_mdy_bcp = bp_filter.column_name_modification(bc_p)
clmn_mdy_bcp

,>ENSP00000400646 (Delta: -0.3486),>ENSP00000410242 (Delta: -0.2813),>ENSP00000477443 (Delta: -0.0776),>ENSP00000357459 (Delta: 0.0759),>ENSP00000340796 (Delta: 0.0604),>ENSP00000337736 (Delta: 0.1470),>ENSP00000265085 (Delta: -0.0539),>ENSP00000445508 (Delta: 1.0364),>ENSP00000358140 (Delta: 0.0479),>ENSP00000480140 (Delta: 0.0065),>ENSP00000257552 (Delta: 0.4922)
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,-0.0315,-0.1454,-0.0027,-0.0068,-0.0143,0.0024,0.0343,0.0805,0.0075,0.0223,-0.0006
2,0.0045,-0.0250,0.0028,0.0006,-0.0089,-0.0055,-0.0057,0.0112,0.0027,0.0094,0.0051
3,0.0053,-0.0079,-0.0001,0.0011,0.0011,-0.0065,-0.0015,0.0041,0.0067,-0.0009,0.0060
4,0.0144,-0.0262,0.0103,0.0006,-0.0104,-0.0030,-0.0008,0.0133,0.0017,-0.0024,0.0063
...,...,...,...,...,...,...,...,...,...,...,...
1021,100.0000,100.0000,100.0000,0.0002,100.0000,100.0000,100.0000,100.0000,-0.0023,100.0000,100.0000
1022,100.0000,100.0000,100.0000,0.0027,100.0000,100.0000,100.0000,100.0000,0.0024,100.0000,100.0000
1023,100.0000,100.0000,100.0000,0.0022,100.0000,100.0000,100.0000,100.0000,0.0000,100.0000,100.0000
1024,100.0000,100.0000,100.0000,-0.0002,100.0000,100.0000,100.0000,100.0000,0.0006,100.0000,100.0000


In [46]:
Right_predicted_RNA_binding = pd.DataFrame()

for i in range(1,11):
    kind_of_work = 'RBP'
    iteration = i
    Data_path = f"/Users/csb/Desktop/{kind_of_work}_captum/{kind_of_work}{iteration}_attribution_score.csv"
    Binding_protein_path = "/Users/csb/Desktop/Project/DNA-RNA-binding/RNA-DNA machine learning/DNA-RNA-BP-ProteinFASTA/351_RNAbinding_Protein.txt"
    predict_result_path = f'/Users/csb/Desktop/{kind_of_work}_captum/result_of_{kind_of_work}{iteration}.csv'
    
    bp_filter = binding_protein_filter(Data_path, Binding_protein_path,predict_result_path)
    b_p = bp_filter.binding_protein_filter()
    bc_p = bp_filter.predict_right_filter(b_p)
    clmn_mdy_bcp = bp_filter.column_name_modification(bc_p)
    Right_predicted_RNA_binding = pd.concat([Right_predicted_RNA_binding, clmn_mdy_bcp],axis=1)
Right_predicted_RNA_binding.to_csv('/Users/csb/Desktop/Right_predicted_1to10RNA_binding.csv',index=None)

In [ ]:
-----------------------

In [ ]:
kind_of_work = 'RBP'
iteration = 1
Data_path = f"/Users/csb/Desktop/{kind_of_work}_captum/{kind_of_work}{iteration}_attribution_score.csv"
Binding_protein_path = "/Users/csb/Desktop/Project/DNA-RNA-binding/RNA-DNA machine learning/DNA-RNA-BP-ProteinFASTA/351_RNAbinding_Protein.txt"
predict_result_path = f'/Users/csb/Desktop/{kind_of_work}_captum/result_of_{kind_of_work}{iteration}.csv'

In [ ]:
# binding-protein filter
import re

# binding_protein 전체 리스트
Dt = pd.read_csv(Data_path,index_col=0)
RBP = pd.read_csv(Binding_protein_path,header=None)
RBP = RBP.iloc[::2,0,]
RBP = RBP.reset_index(drop=True)

# 위에서 불러온 파일 중 ENSP 부분 필터링 후 저장
column_list = list(Dt.columns)
extracted_ids = [re.search(r'(>ENSP\d+)', item).group(1) for item in column_list]

RBP_binding = []
for i in RBP:
    for j in extracted_ids:
        if j in i:
            RBP_binding.append(i)

# 불러온 파일 중 binding_protein만 남김
ok_list = []
for l in range(len(Dt.columns)):
    for k in RBP_binding:
        if k in Dt.columns[l]:
            ok_list.append(Dt.columns[l])

binding_protein = Dt[ok_list]
binding_protein

In [ ]:
import pandas as pd

# CSV 파일에서 데이터프레임 읽기
predict_df = pd.read_csv(predict_result_path)

# 필터링된 결과를 저장할 리스트 초기화
keep_list = []

# 데이터프레임을 순회하면서 조건에 맞는 데이터 추출
for i in range(len(predict_df)):
    if predict_df.loc[i, 'class'] == 1 and predict_df.loc[i, 'class'] == predict_df.loc[i, 'prediction']:
        # 리스트 대신 값 자체를 keep_list에 추가
        keep_list.append(predict_df.loc[i, 'ID'])



In [ ]:
binding_correct_protein = pd.DataFrame()

for j in range(len(binding_protein.columns)):
    for k in keep_list:
        if k in binding_protein.columns[j]:
            binding_correct_protein = pd.concat([binding_correct_protein, binding_protein.iloc[:,j]],axis=1)

binding_correct_protein

In [ ]:
# 데이터 프레임을 보기 좋은 형태로 변경
new_data = []

# 리스트의 각 요소를 순회하면서 정규 표현식을 사용하여 변환
for item in binding_correct_protein:
    # 'tensor' 표현을 제거하고 Delta 값을 추출
    new_item = re.sub(r'tensor\(\[([-\d.]+)\], dtype=torch.float64\)', r'\1', item)
    # 중복된 'Delta:' 표현을 하나로 줄임
    new_item = re.sub(r'\(Delta: \(Delta: ([-\d.]+)\)\)', r'(Delta: \1)', new_item)
    new_data.append(new_item)

binding_correct_protein.columns = new_data
binding_correct_protein

In [ ]:
# K-mer greedy search
# Importance score cutoff

# 첫번째 sequence에 대해 실행
sec1 = binding_correct_protein.iloc[:, 0]
for i in range(len(sec1)):
    if int(sec1[i]) == 100:
        sec1 = sec1.drop(i,axis=0)
        
# 결과를 저장할 리스트
greedy_results = []

# 윈도우 크기 1부터 50까지 반복
for window_size in range(1, 51):
    # 슬라이딩 윈도우 수행
    for start in range(len(sec1) - window_size + 1):
        window = sec1[start:start + window_size]
        # 윈도우 내 모든 값의 합 계산
        total = window.sum()
        # 합이 0.01 * 윈도우 크기보다 큰지 확인
        if total > 0.005 * window_size:
            # 조건을 만족하는 경우, 결과 저장
            greedy_results.append({'Window size': window_size, 'Start_Row number': start})

# 결과를 DataFrame으로 변환
greedy_result_df = pd.DataFrame(greedy_results)

# 결과 출력
greedy_result_df

In [ ]:
>ENSP00000387911 (Delta: tensor([0.2092], dtype=torch.float64))
>ENSP00000436580 (Delta: tensor([0.2864], dtype=torch.float64))